In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sdr_plot_backend.signal_utils import PeakDetector  # Replace with actual import path
from sdrfly.sdr.sdr_generic import SDRGeneric

def sweep_and_detect_peaks(start_freq, stop_freq, sample_rate, bandwidth, sdr_type="hackrf", averaging_count=30):
    fft_size = 1024 * 8
    sdr = SDRGeneric(sdr_type, center_freq=start_freq, sample_rate=sample_rate, bandwidth=bandwidth, gain=30, size=fft_size)
    sdr.start()

    detector = PeakDetector(sdr=sdr, averaging_count=averaging_count)

    full_fft = np.zeros(fft_size)
    current_freq = start_freq

    while current_freq <= stop_freq:
        # Receive data once (no thread)
        detector.receive_data(once=True)

        # Detect peaks using the latest FFT data
        detected_peaks = detector.detect_signal_peaks(
            center_freq=current_freq,
            sample_rate=sample_rate,
            fft_size=fft_size,
            min_peak_distance=80,
            threshold_offset=3
        )

        # Accumulate FFT results for plotting
        with detector.fft_lock:
            if detector.fft_results:
                full_fft += np.mean(detector.fft_results, axis=0)

        # Move to the next frequency
        current_freq += bandwidth
        sdr.set_frequency(current_freq)

    sdr.stop()

    # Average the accumulated FFT data
    averaged_fft_db = full_fft / ((stop_freq - start_freq) / bandwidth + 1)

    # Plot the results in dark mode
    frequencies = np.linspace(start_freq, stop_freq, fft_size)

    plt.figure(figsize=(10, 6))
    plt.plot(frequencies / 1e6, averaged_fft_db, color='yellow')
    for peak in detected_peaks:
        plt.axvline(peak['frequency'] / 1e6, color='red', linestyle='--')
        plt.text(peak['frequency'] / 1e6, peak['power'], f'{peak["frequency"] / 1e6:.2f} MHz\n{peak["power"]:.2f} dB', color='white')
    plt.title("Detected Peaks")
    plt.xlabel("Frequency (MHz)")
    plt.ylabel("Magnitude (dB)")
    plt.grid(True, color='gray')
    plt.gca().set_facecolor('black')
    plt.show()

    # Print out the detected peaks and their power levels
    print("Detected peaks:")
    for peak in detected_peaks:
        print(f"Frequency: {peak['frequency'] / 1e6:.2f} MHz, Power: {peak['power']:.2f} dB")

# Sweep settings
start_freq = 80e6  # Start frequency in Hz
stop_freq = 150e6  # Stop frequency in Hz
sample_rate = 60e6  # Sample rate in Hz
bandwidth = 60e6  # Bandwidth in Hz

# Perform the sweep and detect peaks
sweep_and_detect_peaks(start_freq, stop_freq, sample_rate, bandwidth)


/root/sdrfly


SKIQ[402499]: <WARNING> Process 401991 had owned lock but no longer exists, forcing unlock
SKIQ[402499]: <INFO> libsidekiq v4.18.1 (g58be05ffb)
python3[402499]: [card-0] <INFO> Sidekiq card is serial number=7G2Y, hardware M2 C (rev C), product SKIQ-M2-002 (M.2) (part ES01420*-C0-00)
python3[402499]: [card-0] <INFO> Firmware v2.9
python3[402499]: [card-0] <INFO> Sidekiq: FPGA v3.18.1, (date 24052819, FIFO size 16k)
python3[402499]: [card-0] <INFO> Sidekiq card is configured for an internal reference clock
python3[402499]: [card-0] <INFO> Loading calibration data for Sidekiq M.2
python3[402499]: [card-0] <INFO> RF IC version 5.5.1
python3[402499]: [card-0] <INFO> Number of tx channels supported 1, number of rx channels supported 1
python3[402499]: [card-0] <INFO> Default warp voltage not currently stored (user)
[INFO] Using format CF32.
[INFO] Starting RX Sidekiq Thread


: 